In [2]:
%pip install utils

In [3]:
import os
import IPython.display as ipd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl
import sklearn.utils, sklearn.preprocessing, sklearn.decomposition, sklearn.svm
import librosa
import librosa.display
import utils

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
from google.colab import drive 
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [5]:
#Load metadata and features

tracks = pd.read_csv('/content/gdrive/My Drive/APS360/fma_metadata/tracks.csv')
genres = pd.read_csv('/content/gdrive/My Drive/APS360/fma_metadata/genres.csv')
features = pd.read_csv('/content/gdrive/My Drive/APS360/fma_metadata/features.csv')
echonest = pd.read_csv('/content/gdrive/My Drive/APS360/fma_metadata/echonest.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,5,6,8,12,18,20,21,22,24,33,34,38,39,44,47,49) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,

In [6]:
#Confirm arrays are the right size 
np.testing.assert_array_equal(features.index,tracks.index)
assert echonest.index.isin(tracks.index).all()

In [7]:
#Print shapes
tracks.shape, genres.shape, features.shape, echonest.shape

((106576, 53), (163, 5), (106576, 519), (13132, 250))

In [8]:
#Display first 3 rows  
print(tracks[:3])

  Unnamed: 0     album              album.1  ...   track.17 track.18 track.19
0        NaN  comments         date_created  ...  publisher     tags    title
1   track_id       NaN                  NaN  ...        NaN      NaN      NaN
2          2         0  2008-11-26 01:44:45  ...        NaN       []     Food

[3 rows x 53 columns]


In [9]:
#Select small subset
small = tracks[tracks['set.1'] == 'small']
small.shape

(8000, 53)

In [10]:
print('{} top-level genres'.format(len(genres['top_level'].unique())))
genres.reindex(genres['top_level'].unique()).sort_values('#tracks',ascending=False)

16 top-level genres


,genre_id,#tracks,parent,title,top_level
14,15.0,34413.0,0.0,Electronic,15.0
9,10.0,13845.0,0.0,Pop,10.0
20,21.0,8389.0,0.0,Hip-Hop,21.0
17,18.0,5913.0,1235.0,Soundtrack,1235.0
3,4.0,4126.0,0.0,Jazz,4.0
4,5.0,4106.0,0.0,Classical,5.0
8,9.0,1987.0,0.0,Country,9.0
2,3.0,1752.0,0.0,Blues,3.0
13,14.0,1499.0,0.0,Soul-RnB,14.0
5,6.0,914.0,38.0,Novelty,38.0


In [11]:
genres.sort_values('#tracks').head(10)

,genre_id,#tracks,parent,title,top_level
84,175,0,86,Bollywood,2
87,178,0,4,Be-Bop,4
119,377,1,19,Deep Funk,14
82,173,4,86,N. Indian Traditional,2
135,493,4,651,Western Swing,9
118,374,9,20,Banter,20
154,808,12,46,Salsa,2
83,174,17,86,South Indian Traditional,2
132,465,18,20,Musical Theater,20
85,176,23,2,Pacific,2
